## 設定

In [1]:
import os
import sys
import time
import logging
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
import itertools
from pathlib import Path


import torch
import torch.nn
from torch import optim
import torch.nn.functional as F

from data import Data

from primal_net import PrimalNet
from primal_loss import *
from primal_train import *

#from dual_net import DualNet
#from dual_loss import *
#from dual_train import *

import torch
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(str(Path("examine_cases.ipynb").resolve().parent.parent))


In [15]:
device = "mps" #if torch.cuda.is_available() else "cpu"
lambd = np.ones((3,3))*0.001
# lambd = cfg.lambd

cfg = HParams(num_agents = 3,
              device = device,
              lambd = lambd,
              rho = 0.1,
              lagr_iter = 100,
              batch_size = 128,
              epochs = 10000)

cfg.lr = 1e-4

model = PrimalNet(cfg)
model.load_state_dict(torch.load('model_1222.pth'))
model.to(device)
model.eval()

/var/folders/3s/sh8zstl54dl_t2y55d7th57m0000gn/T/ipykernel_60303/2577132240.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_1222

PrimalNet(
  (input_block): Sequential(
    (0): Linear(in_features=18, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
  )
  (layer_out): Linear(in_features=256, out_features=9, bias=True)
)

In [16]:
device

'mps'

In [17]:
import random
random.seed(314)

In [18]:
from utils import normalize_tuples, apply_features, convert_to_float

In [19]:
from primal_loss import compute_t
from efficiency_loss import compute_efficiency_loss
from utils import da_with_t

## パターン1

In [20]:
one_tie_2_1 = list(set(itertools.permutations([2, 2, 1])))
no_tie_patterns = list(itertools.permutations([3, 2, 1]))

one_tie_2_1 = normalize_tuples(one_tie_2_1)
no_tie_patterns = normalize_tuples(no_tie_patterns)
preference_list_p = list(itertools.product(no_tie_patterns, repeat=3))
preference_list_q = list(itertools.product(one_tie_2_1, repeat=3))
pairs = [(random.choice(preference_list_p), random.choice(preference_list_q)) for _ in range(1000)]
df_1 = pd.DataFrame(pairs, columns=['p', 'q'])

In [21]:
df_1 = apply_features(cfg, model, df_1)
df_1 = convert_to_float(df_1)

Processing rows: 100%|██████████| 1000/1000 [01:42<00:00,  9.72it/s]
/Users/meina/Github/meina-t/matching_with_dl/utils.py:162: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(try_convert)


In [22]:
df_1.describe()

,model_efficiency_loss,model_stability_loss,model_sp_loss,da_efficiency_loss,da_stability_loss,da_sp_loss
count,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.0
mean,-0.258292,1.520026e-01,0.248798,-0.176250,0.170672,0.0
std,0.738410,8.622905e-02,0.208262,0.321484,0.101627,0.0
min,-2.250152,7.094528e-07,0.000001,-1.250000,0.000000,0.0
25%,-0.912164,8.914764e-02,0.057300,-0.500000,0.088889,0.0
50%,-0.001587,1.498319e-01,0.221915,0.000000,0.177778,0.0
75%,0.005111,2.075212e-01,0.397366,0.000000,0.244444,0.0
max,2.924163,4.031537e-01,0.882955,0.000000,0.450000,0.0


In [23]:
df_1.head()

,p,q,model_efficiency_loss,model_stability_loss,model_sp_loss,da_efficiency_loss,da_stability_loss,da_sp_loss
0,"((0.5, 0.16666666666666666, 0.3333333333333333...","((0.4, 0.2, 0.4), (0.4, 0.2, 0.4), (0.4, 0.4, ...",0.086036,0.209564,0.355142,0.0,0.200000,0.0
1,"((0.5, 0.3333333333333333, 0.16666666666666666...","((0.2, 0.4, 0.4), (0.4, 0.2, 0.4), (0.4, 0.4, ...",0.000292,0.133355,0.002530,0.0,0.133333,0.0
2,"((0.5, 0.3333333333333333, 0.16666666666666666...","((0.4, 0.4, 0.2), (0.2, 0.4, 0.4), (0.4, 0.4, ...",-0.002414,0.310892,0.486120,0.0,0.361111,0.0
3,"((0.16666666666666666, 0.5, 0.3333333333333333...","((0.4, 0.2, 0.4), (0.2, 0.4, 0.4), (0.2, 0.4, ...",-0.997764,0.222454,0.150245,-0.5,0.266667,0.0
4,"((0.5, 0.16666666666666666, 0.3333333333333333...","((0.4, 0.2, 0.4), (0.4, 0.2, 0.4), (0.2, 0.4, ...",-0.999738,0.111141,0.156208,-0.5,0.177778,0.0


In [24]:
# n行目を取り出したい場合は3の部分をnに変更する
p = df_1['p'][3]
q = df_1['q'][3]

## パターン2

In [ ]:
one_tie_2_2 = list(set(itertools.permutations([2, 1, 1])))
no_tie_patterns = list(itertools.permutations([3, 2, 1]))
one_tie_2_2 = normalize_tuples(one_tie_2_2)
no_tie_patterns = normalize_tuples(no_tie_patterns)
preference_list_p = list(itertools.product(no_tie_patterns, repeat=3))
preference_list_q = list(itertools.product(one_tie_2_2, repeat=3))
pairs = [(random.choice(preference_list_p), random.choice(preference_list_q)) for _ in range(1000)]
df_2 = pd.DataFrame(pairs, columns=['p', 'q'])

In [ ]:
df_2 = apply_features(cfg, model, df_2)
df_2 = convert_to_float(df_2)

In [ ]:
df_2.describe()

In [ ]:
df_2.head()

In [ ]:
# n行目を取り出したい場合は3の部分をnに変更する
p = df_2['p'][3]
q = df_2['q'][3]

## パターン3

In [ ]:
full_tie_pattern = [(1, 1, 1)]
no_tie_patterns = list(itertools.permutations([3, 2, 1]))

full_tie_pattern = normalize_tuples(full_tie_pattern)
no_tie_patterns = normalize_tuples(no_tie_patterns)
preference_list_p = list(itertools.product(no_tie_patterns, repeat=3))
preference_list_q = list(itertools.product(full_tie_pattern, repeat=3))
pairs = [(random.choice(preference_list_p), random.choice(preference_list_q)) for _ in range(1000)]
df = pd.DataFrame(pairs, columns=['p', 'q'])

In [ ]:
df = apply_features(cfg, model, df)
df = convert_to_float(df)

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
p = df['p'][3]
q = df['q'][3]

## 個別のケース

In [ ]:
#　選好度を指定したい場合。！こちらの場合は次のブロックを実行しないように
p = torch.tensor([[[0.6, 0.3, 1], [1, 0.5, 0], [0.3, 0, 1]]]).to(device)
q = torch.tensor([[[0.5, 1.0, 0], [0, 1, 0.5], [0, 0.5, 1.0]]]).to(device)


In [25]:
# 抜き出した行を使用する場合
p = torch.tensor([p]).to(device)
q = torch.tensor([q]).to(device)

In [26]:
model_output = model(p, q)
model_efficiency_loss = compute_efficiency_loss(cfg, model_output, p, q)
model_stability_loss = compute_t(model_output, p, q).mean()
model_sp_loss = compute_spv_w(cfg, model, model_output, p, q).mean()

da_output = da_with_t(p, q)
da_efficiency_loss = compute_efficiency_loss(cfg, da_output, p, q)
da_stability_loss = compute_t(da_output, p, q).mean()
da_sp_loss = compute_spv_w(cfg, da_with_t, da_output, p, q).mean()

In [27]:
print(f"p:{p}")
print(f"q:{q}")
print("\nModel Results:")
print(f"  Output: {model_output}")
print(f"  Efficiency Loss: {model_efficiency_loss}")
print(f"  Stability Loss: {model_stability_loss}")
print(f"  SP Loss: {model_sp_loss}")

print("\nDA Results:")
print(f"  Output: {da_output}")
print(f"  Efficiency Loss: {da_efficiency_loss}")
print(f"  Stability Loss: {da_stability_loss}")
print(f"  SP Loss: {da_sp_loss}")

p:tensor([[[0.1667, 0.5000, 0.3333],
         [0.3333, 0.5000, 0.1667],
         [0.1667, 0.5000, 0.3333]]], device='mps:0')
q:tensor([[[0.4000, 0.2000, 0.4000],
         [0.2000, 0.4000, 0.4000],
         [0.2000, 0.4000, 0.4000]]], device='mps:0')

Model Results:
  Output: tensor([[[1.8157e-03, 7.1655e-04, 9.9747e-01],
         [9.9889e-01, 6.8546e-04, 4.1967e-04],
         [9.1691e-05, 9.9869e-01, 1.2192e-03]]], device='mps:0',
       grad_fn=<DivBackward0>)
  Efficiency Loss: -0.9977641701698303
  Stability Loss: 0.2224535197019577
  SP Loss: 0.15024502575397491

DA Results:
  Output: tensor([[[0.5000, 0.0000, 0.5000],
         [0.5000, 0.5000, 0.0000],
         [0.0000, 0.5000, 0.5000]]], device='mps:0')
  Efficiency Loss: -0.5
  Stability Loss: 0.2666666805744171
  SP Loss: 0.0
